<a href="https://colab.research.google.com/github/anyuanay/info212/blob/main/INFO212_Week8_Lecture_Data_Loading_Merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INFO 212: Data Science Programming 1
___

## Week 8: Data Loading and Merging
---
**Agenda:**
- Read and parse tables from HTML files
 - pd.read_html(html_source.text)
- Read and write JSON data
 - json.loads()
 - json.dumps()
 - pd.read_json()
- Combining and merging datasets
 - numpy.concatenate(list of narrays, axis)
 - pandas.concat(list of DataFrames, axis)
 - pd.merge(df1, df2, left_on='key1', right_on='key2', how='outer')
 - pd.merge(left1, right1, left_on='key', right_index=True)
 - left.join(right, on='key')


In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Data
## Accessing data is a necessary first step for data analysis. We are going to be focused on data input and output using pandas, though there are numerous tools in other libraries to help with reading and writing data in various formats. Input and output typically falls into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

# HTML: Web Scraping

### Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files.

### Pandas has a built-in function, read_html, which uses libraries like lxml and Beautiful Soup to automatically parse tables out of HTML files as DataFrame objects. To show how this works, read in an HTML file  from the United States FDIC government agency showing bank failures: (https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/).

```
import requests

target_url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

html_source = requests.get(target_url)

html_source.status_code

html_source.text

tables = pd.read_html(html_source.text)

```

## Exercise:
Run and test the above code.

### Explore the tables.

### Change the column labels
```
df.columns = ['Bank', 'City', 'State', 'Cert',
       'AI', 'ClosingDate', 'Fund']
```

## Exercise:
Run the above code.

### Parse the datetime column
```
close_timestamps = pd.to_datetime(df['ClosingDate'])
```

### Extract some interesting data
```
close_timestamps.dt.year.value_counts()

close_timestamps.dt.year.value_counts().sort_index()
```

## Exercise:
Run the above code.

# JSON Data
## JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example.

```


obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
```

Create a DataFrame:
```
pd.DataFrame([obj])
```

## How to parse a JSON structure into a Python object?
```
import json

json.loads(obj)

pd.DataFrame([json.loads(obj)])

result = json.loads(obj)

result['siblings']

```

## Exercise:
Run and test the above code.

## Extract the nested information and create a flatten DataFrame.
```
sibs = pd.DataFrame(result['siblings'])

rows = []
for idx, row in sibs.iterrows():
    pets = row['pets']
    for pet in pets:
        cols = {}
        cols['name'] = row['name']
        cols['age']= row['age']
        cols['pet'] = pet
        rows.append(cols)

pd.DataFrame(rows)
```

## Exercise:
Run and test the above code.

## How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dicts (which were previously JSON objects) to the DataFrame constructor and select a subset of the data fields. For example, we did
```
result_df = pd.DataFrame(result['siblings'])

siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
```

## The pandas `read_json` can automatically convert JSON datasets in specific arrangements into a Series or DataFrame. For example, let us load
```
example.json
```

# Combining and Merging Datasets

## Concatenating Along an Axis
## A kind of data combination operation is referred to interchangeably as concatenation, binding, or stacking.

```
df1 = pd.DataFrame([{'name':'Alice', 'Subject':'math', 'Score':95}, {'name':'Bob', 'Subject':'English', 'Score':96}])

df2 = pd.DataFrame([['Alice', 'Frank', 'Charlie'], [19, 20, 18], ['alice@drexel', 'frank@drexel', 'charlie@drexel']])

df2 = df2.T

df2.columns = ['name', 'age', 'email']

df3  = pd.concat([df1, df2], axis=1, keys=['df1', 'df2'])

pd.concat([df1, df2])

```

## Exercise:
Run and test the above code.

## In the context of pandas objects such as Series and DataFrame, having labeled axes enable you to further generalize array concatenation. In particular, you have a number of additional things to think about:
* If the objects are indexed differently on the other axes, should we combine the
distinct elements in these axes or use only the shared values (the intersection)?
* Do the concatenated chunks of data need to be identifiable in the resulting
object?
* Does the “concatenation axis” contain data that needs to be preserved? In many
cases, the default integer labels in a DataFrame are best discarded during
concatenation.

## The concat function in pandas provides a consistent way to address each of these concerns. Here is a list of examples to illustrate how it works. Suppose we have three Series with no index overlap:
```
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
```

## Calling concat with these objects in a list glues together the values and indexes:
```
pd.concat([s1, s2, s3], axis = 1)
```

## By default concat works along axis=0, producing another Series. If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):
```
pd.concat([s1, s2, s3])
```

## In this case there is no overlap on the other axis, which as you can see is the sorted union (the 'outer' join) of the indexes. You can instead intersect them by passing join='inner':
```
s4 = pd.concat([s1, s3])
pd.concat([s1, s4], axis=1, join='inner')
```

### Combine DataFrames:
We can concatenate dataframes on axis to combine them vertically or horizontally.
```
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

pd.concat([df1, df2], axis=0)

pd.concat([df1, df2], axis=1)
```

## Exercise:
Run and test the above code.

## Database-Style DataFrame Joins
## Merge or join operations combine datasets by linking rows using one or more keys. These operations are central to relational databases (e.g., SQL-based). The merge function in pandas is the main entry point for using these algorithms on your data. Let’s start with a simple example:

```
df1 = pd.DataFrame({'key1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key2': ['a', 'b', 'd'],
                    'data2': range(3)})

pd.merge(df1, df2, left_on='key1', right_on='key2', how='outer')

df1.merge(df2, left_on='key1', right_on='key2')

df1.join(df2)
```

## Exercise:
Run and test the above code.

## Note that I didn't specify which column to join on. If that information is not specified, merge uses the overlapping column names as the keys. It's a good practice to specify explicitly, though:
```
pd.merge(df1, df2, left_on='key', right_on='key')
```

## By default merge does an 'inner' join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are 'left', 'right', and 'outer'. The outer join takes the union of the keys, combining the effect of applying both left and right joins:
```
pd.merge(df1, df2)

pd.merge(df1, df2, how='outer')
```

## Merging on Index
## In some cases, the merge key(s) in a DataFrame will be found in its index. In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key:

```
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

pd.merge(left1, right1.reset_index(), left_on='key', right_on='index')

pd.merge(left1, right1, left_on='key', right_index=True)


```

## Exercise:
Run and test the above code.